The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import chi2
import statsmodels.discrete.discrete_model as dm_
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sys.path.insert(0, os.path.abspath('../'))

%matplotlib inline

from lec1.preprocessor import *
from lec1.feature_engineering import *

import toad

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data_path = f"{os.path.abspath('.')}/home-credit-default-risk/"

df_ft = pd.read_pickle(data_path + 'features.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ruofei.ouyang/workspace/services/fal-model-v2/lec1/home-credit-default-risk/features.pkl'

# 模型数据准备

In [61]:
train, test = train_test_split(df_ft, test_size=0.3)

### 根据缺失值占比，iv值，和高相关性进行变量筛选，赋值为：

（1）empyt=0.5: 若变量的缺失值大于0.5被删除

（2）iv=0.02: 若变量的iv值小于0.02被删除

（3）corr=0.7: 若两个相关性高于0.7时，iv值低的变量被删除

（4）return_drop=False: 若为True，function将返回被删去的变量列

（5）exclude=None: 明确不被删去的列名

In [62]:
train_selected, dropped = toad.selection.select(train, target = 'TARGET', empty = 0.5, iv = 0.02, corr = 0.7, return_drop=True, exclude=['SK_ID_CURR'])

/opt/anaconda3/lib/python3.7/site-packages/toad/selection.py:332: RuntimeWarning: invalid value encountered in greater
  ix, cn = np.where(np.triu(corr.values, 1) > threshold)


In [63]:
train_selected.to_pickle(data_path + 'selected_train.pkl')

In [65]:
to_drop = ['SK_ID_CURR']
c = toad.transform.Combiner()
c.fit(train_selected.drop(to_drop, axis=1), y = 'TARGET', method = 'chi', min_samples = 0.05) #empty_separate = False



In [66]:
transer = toad.transform.WOETransformer()

train_woe = transer.fit_transform(c.transform(train_selected), train_selected['TARGET'], exclude=to_drop+['TARGET'])

print(train_woe.head(3))

        SK_ID_CURR  TARGET  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  DAYS_ID_PUBLISH  REGION_RATING_CLIENT_W_CITY  REG_CITY_NOT_LIVE_CITY  REG_CITY_NOT_WORK_CITY  EXT_SOURCE_2  EXT_SOURCE_3  YEARS_BEGINEXPLUATATION_AVG  FLOORSMAX_MEDI  TOTALAREA_MODE  DAYS_LAST_PHONE_CHANGE  FLAG_DOCUMENT_3  AMT_REQ_CREDIT_BUREAU_YEAR  CODE_GENDER_M  NAME_INCOME_TYPE_Pensioner  NAME_INCOME_TYPE_Working  NAME_EDUCATION_TYPE_Higher education  OCCUPATION_TYPE_Laborers  OCCUPATION_TYPE_nan  EMERGENCYSTATE_MODE_No  PAYMENT_RATE  id_renewal_days_delay  b_DAYS_CREDIT  b_CREDIT_DAY_OVERDUE  b_DAYS_CREDIT_ENDDATE  b_AMT_CREDIT_MAX_OVERDUE  b_AMT_CREDIT_SUM  b_AMT_CREDIT_SUM_DEBT  b_AMT_CREDIT_SUM_LIMIT  b_AMT_CREDIT_SUM_OVERDUE  b_CREDIT_ACTIVE_Closed  b_CREDIT_TYPE_Car loan  b_CREDIT_TYPE_Credit card  b_CREDIT_TYPE_Microloan  b_CREDIT_TYPE_Mortgage  SK_ID_PREV    SK_DPD  SK_DPD_DEF  pos_cash_paid_late_12_cnt  i_NUM_INSTALMENT_VERSION  i_NUM_INSTALMENT_NUMBER  \

In [69]:
train_woe.to_pickle(data_path + 'train_woe.pkl')

In [72]:
test_woe = transer.transform(c.transform(test[train_selected.columns]))


In [73]:
test_woe.to_pickle(data_path + 'test_woe.pkl')

# 前向特征筛选

In [75]:
final_data = toad.selection.stepwise(train_woe, target = 'TARGET', estimator='lr', direction = 'forward', criterion = 'ks', exclude = to_drop)

# 将选出的变量应用于test数据
final_test = test_woe[final_data.columns]

print(final_data.shape)


(215257, 6)


In [76]:
final_data.to_pickle(data_path + 'final_data.pkl')